In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
from configparser import ConfigParser
import unidecode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
import requests

In [ ]:
fake_headers = {'Host': 'www.spar.hu',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
'Accept-Language': 'hu-HU,hu;q=0.8,en-US;q=0.5,en;q=0.3',
'Accept-Encoding': 'gzip, deflate, br',
'Connection': 'keep-alive',
'Upgrade-Insecure-Requests': '1',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'none',
'Sec-Fetch-User': '?1',
'TE': 'trailers'}

In [2]:
def get_all_link_spar():
    offer_page = 'https://www.spar.hu/onlineshop/'

    r = requests.get(offer_page)
    soup = bs(r.content)
    a_list = soup.body.findAll('a', {'class' : ['flyout-categories__link']})
    all_link = set()
    for a in a_list:
        if a['href'] != 'javascript:void(0)':
            ref = a['href']
            #print(ref)
            base_string = 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:'
            categ = ref[ref.find('/c/'):].replace('/c/','').replace('/','')
            all_link.add(base_string + categ)
            #all_link.append('https://www.spar.hu/onlineshop'+a['href']+'?query=*&q=*&hitsPerPage=800')
    
    return list(all_link)

In [3]:
all_link = get_all_link_spar()

In [4]:
all_link

['https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H6-1-1',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-9',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-2',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-2-3',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H14-5',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H2-6-5',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H7-1-6',
 'https://search-spar.spar-ics.com/fact

In [5]:
fake_headers = {'Host': 'search-spar.spar-ics.com',
               'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0',
               'Accept': 'application/json, text/javascript, */*; q=0.01',
               'Accept-Language': 'hu-HU,hu;q=0.8,en-US;q=0.5,en;q=0.3',
               'Accept-Encoding': 'gzip, deflate, br',
               'Origin': 'https://www.spar.hu',
               'Connection': 'keep-alive',
               'Referer': 'https://www.spar.hu/',
               'Sec-Fetch-Dest': 'empty',
               'Sec-Fetch-Mode': 'cors',
               'Sec-Fetch-Site': 'cross-site',
               'TE': 'trailers'}

In [6]:
url = 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-5-2'

In [7]:
response = requests.get(url, headers=fake_headers)

In [8]:
response.json()['hits']

[{'masterValues': {'category-names': 'Tejdesszertek|Tejszeletek, túró-rudi|Tejtermék, sajt, tojás, kényelmi termékek',
   'badge-icon': '/onlineshop/medias/sys_master/images/images/h0c/hab/9444906631198/chilled-v-gleges.png',
   'description': 'PÖTTY.TÚRÓ RUDI 10*30G',
   'sales-unit': 'db',
   'title': 'Pöttyös Túró Rudi natúr túródesszert 10 x 30 g (300 g)',
   'sales': '26330',
   'badge-names': 'Hűtött',
   'was-price-per-unit': '4 966.67 Ft/kg',
   'item-type': 'Product',
   'code-internal': '8796333015041',
   'price': 1490.0,
   'created-at': '1526609249891',
   'categories': ['H3_Tejtermék, sajt, tojás, kényelmi termékek:H3-5_Tejdesszertek',
    'H3-5_Tejdesszertek:H3-5-2_Tejszeletek, túró-rudi',
    'root_null:H3_Tejtermék, sajt, tojás, kényelmi termékek'],
   'best-price': 1490.0,
   'stock-status': 'inStock',
   'is-new': 'false',
   'image-url': 'https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/355820001/dt_main.jpg',
   'category-id': 'H3-5-2|1104999999|H3-5|

In [9]:
all_items = []
counter = 1
for url in all_link:
    response = requests.get(url, headers=fake_headers)
    print(f'crawl url: {url} done {counter} from {len(all_link)}')
    counter = counter + 1
    for hit in response.json()['hits']:
        
        item_dict = {}
        item_dict['itemId'] = hit['masterValues']['product-number']
        item_dict['itemName'] = hit['masterValues']['title']
        item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()
        item_dict['imageUrl'] = hit['masterValues']['image-url']
        item_dict['price'] = hit['masterValues']['price']
        item_dict['measure'] = hit['masterValues']['sales-unit']
        item_dict['salesStart'] = np.nan
        item_dict['source'] = 'spar'
        item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
        item_dict['shopName'] = 'spar'
        if (hit['masterValues']['regular-price'] != hit['masterValues']['price']):
            item_dict['isSales'] = 1
        else:
            item_dict['isSales'] = 0
            
        
        all_items.append(item_dict)

df = pd.DataFrame(all_items)

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H6-1-1 done 1 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-9 done 2 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-2 done 3 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-2-3 done 4 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H14-5 done 5 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H2-6-5 done 6 from 565
crawl url: https://search-spar.spar-ics

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H2-1-1 done 53 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-3-4 done 54 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H12-2-4 done 55 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H24-1 done 56 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-14-3 done 57 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-1-7 done 58 from 565
crawl url: https://search-s

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H12-6-13 done 104 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-5-6 done 105 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H17-3-3 done 106 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H17-4-1 done 107 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-9 done 108 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-2-1 done 109 from 565
crawl url: https://s

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H25 done 156 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H7-2 done 157 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H11-3-3 done 158 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H11-6-7 done 159 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-4-3 done 160 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-12-4 done 161 from 565
crawl url: https://searc

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9 done 207 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H24-2 done 208 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H1-3-3 done 209 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H12-4 done 210 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-6-3 done 211 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-14 done 212 from 565
crawl url: https://search-spa

...

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H11-3-4 done 311 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-2-6 done 312 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-10-1 done 313 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H8-3 done 314 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H6-7 done 315 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-6-3 done 316 from 565
crawl url: https://searc

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H11-5-3 done 362 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H14 done 363 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H14-1 done 364 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-1-3 done 365 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H1-3-4 done 366 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-10-1 done 367 from 565
crawl url: https://searc

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H12-1 done 414 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H13 done 415 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H14-7-1 done 416 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-2-5 done 417 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-2 done 418 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H12-6-11 done 419 from 565
crawl url: https://search

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-1-1 done 465 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-5-1 done 466 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-7-3 done 467 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-10-4 done 468 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H1-3-2 done 469 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H10-2-4 done 470 from 565
crawl url: https://s

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H17-5 done 516 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-8-2 done 517 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-5-7 done 518 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H4-3-2 done 519 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-9-8 done 520 from 565
crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H14-4-5 done 521 from 565
crawl url: https://sea

In [10]:
df['isSales'].value_counts()

0    43903
1      427
Name: isSales, dtype: int64

In [11]:
df = df.fillna('N.a')

In [12]:
df.loc[~df['price'].str.isnumeric()]

AttributeError: Can only use .str accessor with string values!

In [13]:
df[df['imageUrl'].isnull()]

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales


In [14]:
df.to_excel('spar.xlsx', index=False)